In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch
import torch.nn as nn

In [3]:
from data_preprocessing import process_resstock_data

# Load data

### Setpoint data

In [4]:
# Load full heating and cooling timeseries
with open('/Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/full_heating_timeseries.pkl', 'rb') as f:
    full_heating_timeseries = pickle.load(f)
with open('/Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/full_cooling_timeseries.pkl', 'rb') as f:
    full_cooling_timeseries = pickle.load(f)

In [5]:
heating_timeseries = full_heating_timeseries[1000] # Builiding 1000
cooling_timeseries = full_cooling_timeseries[1000]

In [6]:
heating_timeseries = pd.DataFrame(heating_timeseries)
cooling_timeseries = pd.DataFrame(cooling_timeseries)

In [7]:
heating_timeseries.head()

,setpoint
timestamp,
2018-01-01 00:00:00,76.0
2018-01-01 01:00:00,76.0
2018-01-01 02:00:00,76.0
2018-01-01 03:00:00,76.0
2018-01-01 04:00:00,76.0


### Weather data

In [8]:
weather_data = pd.read_csv("/Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/G2500170_2018.csv")

In [9]:
weather_data.head()

,date_time,Dry Bulb Temperature [°C],Relative Humidity [%],Wind Speed [m/s],Wind Direction [Deg],Global Horizontal Radiation [W/m2],Direct Normal Radiation [W/m2],Diffuse Horizontal Radiation [W/m2]
0,2018-01-01 01:00:00,-17.2,47.00,7.2,310.0,0.0,0.0,0.0
1,2018-01-01 02:00:00,-17.8,46.83,6.7,300.0,0.0,0.0,0.0
2,2018-01-01 03:00:00,-18.9,55.19,2.1,260.0,0.0,0.0,0.0
3,2018-01-01 04:00:00,-18.9,51.97,4.1,280.0,0.0,0.0,0.0
4,2018-01-01 05:00:00,-19.4,54.51,3.6,280.0,0.0,0.0,0.0


### ResStock

In [10]:
resstock = pd.read_parquet("/Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/MA_baseline_metadata_and_annual_results.parquet")

In [11]:
resstock.head()

,upgrade,weight,applicability,in.sqft,in.representative_income,in.ahs_region,in.aiannh_area,in.area_median_income,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.battery,in.bedrooms,in.building_america_climate_zone,in.cec_climate_zone,in.ceiling_fan,in.census_division,in.census_division_recs,in.census_region,in.city,in.clothes_dryer,in.clothes_dryer_usage_level,in.clothes_washer,in.clothes_washer_presence,in.clothes_washer_usage_level,in.cooking_range,in.cooking_range_usage_level,in.cooling_setpoint,in.cooling_setpoint_has_offset,in.cooling_setpoint_offset_magnitude,in.cooling_setpoint_offset_period,in.corridor,in.county,in.county_and_puma,in.county_name,in.dehumidifier,in.dishwasher,in.dishwasher_usage_level,in.door_area,in.doors,...,out.site_energy.total.energy_consumption.kwh,out.hot_water.clothes_washer.gal,out.hot_water.dishwasher.gal,out.hot_water.distribution_waste.gal,out.hot_water.fixtures.gal,out.load.cooling.energy_delivered.kbtu,out.load.heating.energy_delivered.kbtu,out.load.hot_water.energy_delivered.kbtu,out.electricity.summer.peak.kw,out.electricity.winter.peak.kw,out.load.cooling.peak.kbtu_hr,out.load.heating.peak.kbtu_hr,out.unmet_hours.cooling.hour,out.unmet_hours.heating.hour,out.emissions.all_fuels.lrmer_high_re_cost_15.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_25.co2e_kg,out.emissions.electricity.lrmer_high_re_cost_15.co2e_kg,out.emissions.electricity.lrmer_low_re_cost_15.co2e_kg,out.emissions.electricity.lrmer_mid_case_15.co2e_kg,out.emissions.electricity.lrmer_mid_case_25.co2e_kg,out.emissions.fuel_oil.lrmer_high_re_cost_15.co2e_kg,out.emissions.fuel_oil.lrmer_low_re_cost_15.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_25.co2e_kg,out.emissions.natural_gas.lrmer_high_re_cost_15.co2e_kg,out.emissions.natural_gas.lrmer_low_re_cost_15.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_25.co2e_kg,out.emissions.propane.lrmer_high_re_cost_15.co2e_kg,out.emissions.propane.lrmer_low_re_cost_15.co2e_kg,out.emissions.propane.lrmer_mid_case_15.co2e_kg,out.emissions.propane.lrmer_mid_case_25.co2e_kg,out.bills.all_fuels.usd,out.bills.electricity.usd,out.bills.fuel_oil.usd,out.bills.natural_gas.usd,out.bills.propane.usd,out.energy_burden.percentage
bldg_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,0,252.301639,True,1228,268295.0,"CBSA Boston-Cambridge-Newton, MA-NH",No,150%+,5A,5A,Hour17,None,3,Cold,None,Standard Efficiency,New England,New England,Northeast,"MA, Waltham",Electric,80% Usage,EnergyStar,Yes,80% Usage,Electric Induction,80% Usage,65F,No,0F,None,Not Applicable,G2500170,"G2500170, G25000503",Middlesex County,None,290 Rated kWh,80% Usage,20 ft^2,Fiberglass,...,31331.935041,548.7,1313.7,3181.0,12989.8,6337.0,63666.0,10289.0,6.1342,8.0336,5.415,59.387,117.25,0.00,8890.873116,8597.661936,8981.609734,8818.429879,2032.443084,1739.231904,2123.179701,1959.999846,6858.430032,6858.430032,6858.430032,6858.430032,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3661.499038,1991.889038,1669.61,0.00,0.00,1.36
122,0,252.301639,True,623,64407.0,"CBSA Boston-Cambridge-Newton, MA-NH",No,80-100%,5A,5A,Hour23,None,1,Cold,None,Standard Efficiency,New England,New England,Northeast,"MA, Boston",None,100% Usage,None,None,100% Usage,Gas,100% Usage,70F,No,0F,None,Double-Loaded Interior,G2500250,"G2500250, G25003301",Suffolk County,None,318 Rated kWh,100% Usage,20 ft^2,Fiberglass,...,17198.875753,0.0,481.1,1699.2,4757.2,7827.0,29495.0,3832.0,3.5342,3.4146,8.650,22.743,111.50,43.50,3970.062683,3782.579348,4014.523807,3916.561462,1258.283378,1070.800044,1302.744502,1204.782158,0.000000,0.000000,0.000000,0.000000,2711.779304,2711.779304,2711.779304,2711.779304,0.000000,0.000000,0.000000,0.000000,1911.148724,1216

### Building timeseries

In [12]:
building_data = pd.read_parquet("/Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/1000-0.parquet")

In [13]:
building_data.head()

,timestamp,out.electricity.ceiling_fan.energy_consumption,out.electricity.ceiling_fan.energy_consumption_intensity,out.electricity.clothes_dryer.energy_consumption,out.electricity.clothes_dryer.energy_consumption_intensity,out.electricity.clothes_washer.energy_consumption,out.electricity.clothes_washer.energy_consumption_intensity,out.electricity.cooling.energy_consumption,out.electricity.cooling.energy_consumption_intensity,out.electricity.cooling_fans_pumps.energy_consumption,out.electricity.cooling_fans_pumps.energy_consumption_intensity,out.electricity.dishwasher.energy_consumption,out.electricity.dishwasher.energy_consumption_intensity,out.electricity.freezer.energy_consumption,out.electricity.freezer.energy_consumption_intensity,out.electricity.heating.energy_consumption,out.electricity.heating.energy_consumption_intensity,out.electricity.heating_fans_pumps.energy_consumption,out.electricity.heating_fans_pumps.energy_consumption_intensity,out.electricity.heating_hp_bkup.energy_consumption,out.electricity.heating_hp_bkup.energy_consumption_intensity,out.electricity.heating_hp_bkup_fa.energy_consumption,out.electricity.heating_hp_bkup_fa.energy_consumption_intensity,out.electricity.hot_water.energy_consumption,out.electricity.hot_water.energy_consumption_intensity,out.electricity.lighting_exterior.energy_consumption,out.electricity.lighting_exterior.energy_consumption_intensity,out.electricity.lighting_garage.energy_consumption,out.electricity.lighting_garage.energy_consumption_intensity,out.electricity.lighting_interior.energy_consumption,out.electricity.lighting_interior.energy_consumption_intensity,out.electricity.mech_vent.energy_consumption,out.electricity.mech_vent.energy_consumption_intensity,out.electricity.net.energy_consumption,out.electricity.net.energy_consumption_intensity,out.electricity.permanent_spa_heat.energy_consumption,out.electricity.permanent_spa_heat.energy_consumption_intensity,out.electricity.permanent_spa_pump.energy_consumption,out.electricity.permanent_spa_pump.energy_consumption_intensity,out.electricity.plug_loads.energy_consumption,...,out.site_energy.net.energy_consumption,out.site_energy.net.energy_consumption_intensity,out.site_energy.total.energy_consumption,out.site_energy.total.energy_consumption_intensity,out.load.cooling.energy_delivered.kbtu,out.load.heating.energy_delivered.kbtu,out.load.hot_water.energy_delivered.kbtu,out.outdoor_air_dryblub_temp.c,out.zone_mean_air_temp.air_source_heat_pump_airloop_ret_air_zone.c,out.zone_mean_air_temp.attic_unvented.c,out.zone_mean_air_temp.attic_vented.c,out.zone_mean_air_temp.basement_unconditioned.c,out.zone_mean_air_temp.central_ac_airloop_ret_air_zone.c,out.zone_mean_air_temp.central_ac_and_furnace_airloop_ret_air_zone.c,out.zone_mean_air_temp.conditioned_space.c,out.zone_mean_air_temp.crawlspace_unvented.c,out.zone_mean_air_temp.crawlspace_vented.c,out.zone_mean_air_temp.furnace_airloop_ret_air_zone.c,out.zone_mean_air_temp.garage.c,out.zone_mean_air_temp.ground_source_heat_pump_airloop_ret_air_zone.c,out.total.lrmer_highrecost_15__electricity.co2e_kg,out.total.lrmer_highrecost_15__fuel_oil.co2e_kg,out.total.lrmer_highrecost_15__natural_gas.co2e_kg,out.total.lrmer_highrecost_15__propane.co2e_kg,out.total.lrmer_highrecost_15.co2e_kg,out.total.lrmer_lowrecost_15__electricity.co2e_kg,out.total.lrmer_lowrecost_15__fuel_oil.co2e_kg,out.total.lrmer_lowrecost_15__natural_gas.co2e_kg,out.total.lrmer_lowrecost_15__propane.co2e_kg,out.total.lrmer_lowrecost_15.co2e_kg,out.total.lrmer_midcase_15__electricity.co2e_kg,out.total.lrmer_midcase_15__fuel_oil.co2e_kg,out.total.lrmer_midcase_15__natural_gas.co2e_kg,out.total.lrmer_midcase_15__propane.co2e_kg,out.total.lrmer_midcase_15.co2e_kg,out.total.lrmer_midcase_25__electricity.co2e_kg,out.total.lrmer_midcase_25__fuel_oil.co2e_kg,out.total.lrmer_midcase_25__natural_gas.co2e_kg,out.total.lrmer_midcase_25__propane.co2e_kg,out.total.lrmer_midcase_25.co2e_kg
bldg_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

# ResStock testing

In [14]:
columns = [
    "in.area_median_income",    # 0-30%, 80-100% ...
    "in.bedrooms",              # int
    "in.duct_leakage_and_insulation",    # Separate into leakage / insulation: X% / (Uniinsulated/R-n)
    "in.duct_location",         # Places of the house
    "in.federal_poverty_level", # 100-150%, 400%+...
    "in.geometry_floor_area",   # 1000-1499, ...
    "in.geometry_stories",      # int
    "in.geometry_wall_type",    # Materials
    "in.ground_thermal_conductivity",   # float
    "in.heating_fuel",      # Fuel type
    "in.hvac_cooling_efficiency",   # Format: AC, (S)EER 10.7
    "in.hvac_has_ducts",    # Yes / No
    "in.hvac_heating_efficiency",   # Format: Heater type, 80% AFUE/EFFICIENCY/HSPF
    "in.income",            # 70000-79999 
    "in.insulation_ceiling",    # R-n / Uninsulated
    "in.insulation_floor",      # R-n / Uninsulated
    "in.insulation_foundation_wall",    # Wall R-n, Exterior
    "in.insulation_roof",       # Finished / Unfinished, R-n
    "in.insulation_wall",       # Material, Uninsulated / R-n
    "in.occupants",         # int
    "in.orientation",       # North, Northwest...
    "in.roof_material",     # Material
    "in.sqft",              # int
    "in.windows"    # Single/Double/Triple, Low-E/Clear, Metal/Non-metal, Air/Exterior Clear Storm, (L/M-Gain)
]

In [23]:
resstock[columns[10]].value_counts() 

in.hvac_cooling_efficiency
Room AC, EER 10.7       3015
Room AC, EER 12.0       2335
AC, SEER 13             1941
None                    1408
AC, SEER 15              848
Room AC, EER 9.8         801
Shared Cooling           404
AC, SEER 10              403
Ducted Heat Pump         244
Room AC, EER 8.5         150
Non-Ducted Heat Pump      91
AC, SEER 8                67
Name: count, dtype: int64

In [16]:
resstock_2 = process_resstock_data(resstock)

In [18]:
resstock_2["in.geometry_floor_area_processed"].value_counts()

in.geometry_floor_area_processed
1749    3131
2499    1761
1249    1726
3249    1426
874     1249
4999     852
3999     664
249      533
4000     365
Name: count, dtype: int64